In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [42]:
class Value:
    
    def __init__(self,data,_children=(),_op=''):
        self.data = data
        self._prev = set(_children)
        self.grad = 0
        self._op = _op
        self._backward = lambda:None
        self.prev = set(_children)
        self._op = _op
        #print(self._prev)
        #print(self.data)
 
    
    def __add__(self, other):
        other = other if isinstance(other,Value) else Value(other)
        out = Value(self.data + other.data,(self,other),_op='+')
        
        def _backward():
            # Addition because k(x) + y(x) = f if this is the case then we are overwriting the gradients that are involved, hence add ( multivariable differentiation) ez
            self.grad += out.grad
            other.grad += out.grad
        
        # ._backward function is assinged to out._backward allowing to call later. We are assigning the function object itself rather than the result of calling the 
        # function.
        out._backward = _backward
        return out
    
    def __mul__(self,other):
        other = other if isinstance(other,Value) else Value(other)
        out = Value(self.data * other.data,(self,other),_op='* ')
        
        def _backward():
            self.grad += out.grad * other.data
            other.grad += self.data * out.grad
        
        out._backward = _backward
        return out  
    
    def __pow__(self, other):
        assert isinstance(other,(int,float)), "Only supporting int/float powers"
        out = Value(self.data ** other,(self,),f'**{other}')
        
        def _backward():
            self.grad += (other*self.data**(other-1))*out.grad
        out._backward= _backward
        
    def relu(self):
        out = Value(0 if self.data <0 else self.data,(self,),'ReLU')
        
        def _backward():
            self.grad += out.grad*(out.data>0)
        
        out._backward = _backward
        return out
    
    def backward(self):
        # Topological ordering of all children
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        
        # go one variable at a time and apply the chain rule to its gradient
        self.grad = 1
        for v in reversed(topo):
            v._backward()

    def __neg__(self): 
        return self*-1
    def __radd__(self, other): # other + self
        return self + other

    def __sub__(self, other): # self - other
        return self + (-other)

    def __rsub__(self, other): # other - self
        return other + (-self)

    def __rmul__(self, other): # other * self
        return self * other

    def __truediv__(self, other): # self / other
        return self * other**-1

    def __rtruediv__(self, other): # other / self
        return other * self**-1
  
    def __repr__(self):
        return f"Value(data={self.data})"
    #Python internally uses this function to return the string
    
a = Value(2.0)
b = Value(-3.0)
c = Value(10.0)

d = a*b + c
d
print(a.grad)

0


In [50]:
d.backward()
print(b.grad)

72.0


In [6]:
def xy():
    print("Hello this is xy")

ab = xy
ab

<function __main__.xy()>